# Versions Python et Libraries

In [2]:
!python --version

Python 3.12.3


In [5]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
# import shutil

In [ ]:
# Modélisation CNN avec tensorflow
import tensorflow as tf
import tensorflow.keras.layers as tfl

# Gestion des images : lecture, transformations
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation

# Gestion de l'architecture du réseau
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Flatten, Conv2D, Activation, Dense, Dropout
from tensorflow.keras.layers import MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Resizing, Rescaling, BatchNormalization
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy

# Architecture de modèles de réseaux pré-entrainés (fonctionnalité de Transfer Learning)
from tensorflow.keras.applications import MobileNetV2, VGG16

# Algorithme d'optimisation
from tensorflow.keras.optimizers import RMSprop, Adam

# Sauvegarde, arret
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

# Open CV
import cv2

# Performances des modeles
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.utils.multiclass import unique_labels

# Constantes

In [ ]:
# chemin des 2 dossiers d'images par classe à prédire pour l'apprentissage et l'évaluation des performances et l'inférence
SRC_PATH_TRAIN = "./dataset_footprint/train/"
SRC_PATH_TEST = "./dataset_footprint/test/"

# Liste des catégories (classes à prévoir)
LST_LABELS = os.listdir(SRC_PATH_TRAIN)

# Parametres pour la generation d'images
SEED_VALUE = 42
VALID_SIZE = 0.2

In [ ]:
IMG_SIZE = 160  # Taille de l'image IMG_SIZExIMG_SIZE (on augmente la resolution de 100 a 160)
BATCH_SIZE = 10  # nb de données à passer pour un A/R dans le réseau (total de 77 images x 3 classes)
NB_EPOCHS = 30  # Nb de passes

In [ ]:
# Définition comme label les noms des sous-dossiers de travail
labels = os.listdir(SRC_PATH_TRAIN)
LST_DIR_LABELS = labels
LST_DIR_LABELS

In [ ]:
# Sauvegarde du modèle optimal (nom et sous-dossier)
CKPT_NO, MDL_NAME = 'ckpt_footprints_1', '3footprints_CNN'
CKPT_DIR = './'+ CKPT_NO

# Fonctions Locales

## Graphs

In [ ]:
def plot_learning_curve(history):
    """
    Fonction de tracé de la courbe d'ajustement d'un modèle
    Arguments:
        history : sequence de recueil des métriques d'évaluation d'un modèle lors de la phase d'apprentissage
        les métriques sont ici prédéfinies : 'accuracy','val_accuracy','loss','val_loss'
    Returns:
        2 figures Matplotlib superposées des métriques 'accuracy' et 'loss' sur les datasets TRAIN et VALIDATION
    """
    # Définition des séquences de sauvegarde des performances en TRAIN (accuracy et loss) et VALIDATION (val_)
    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    # Liste des itérations de calcul : epochs
    lst_epochs = range(len(accuracy))
    
    # Tracé en 2 figures
    plt.figure(figsize=(20,6))
    plt.plot(lst_epochs, accuracy, "b", label="accuracy [TRAIN]")
    plt.plot(lst_epochs, val_accuracy, "r", label="accuracy [VALIDATION]")
    plt.title("Exactitude du modèle")
    plt.legend()
    plt.show()

    plt.figure(figsize=(20,6))
    plt.plot(lst_epochs, loss, "b", label="loss [TRAIN]")
    plt.plot(lst_epochs, val_loss, "r", label="loss [VALIDATION]")
    plt.title("Courbe de perte du modèle")    
    plt.legend()
    plt.show()

In [ ]:
# WARNING THIS FUNCTION IS BASICALLY UNUSED

# create a confusion matrix to visually represent incorrectly classified images
def plot_confusion_matrix(y_true, y_pred, classes, out_path=""):
    cm = confusion_matrix(y_true, y_pred)
    df_cm = pd.DataFrame(cm, index=[i for i in classes], columns=[i for i in classes])
    plt.figure(figsize=(3,3))
    ax = sns.heatmap(df_cm, annot=True, square=True, fmt="d", linewidths=.2, cbar_kws={"shrink": 0.8})
    if out_path:
        plt.savefig(out_path + "/confusion_matrix.png")  # as in the plot_model_history, the matrix is saved in a file called "model_name_confusion_matrix.png"
    return ax

# Définition de l'architecture du réseau de neurones convolutifs (CNN)

In [ ]:
def prepare_model():
    # Initialisation du réseau
    model = Sequential()
    
    # Blocs de Convolution
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3), padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
   
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    # Couches de classification
    model.add(Flatten())

    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.3))
    
    #model.add(Dense(16, activation='relu'))
    #model.add(Dropout(0.3))
    
    # Couche de sortie
    model.add(Dense(len(LST_LABELS), activation='softmax'))
    
    return model

In [ ]:
# Fonction d'inférence avec le modele model de l'image n° numero du dossier src_path de la classe categorie
def f_footprint_predict(categorie, numero, model, src_path=SRC_PATH_TEST, lst_labels=LST_LABELS):
    # Image à classifier dans la catégorie courante
    id_image = categorie + '_' + numero + '.jpg'
    lb_image = src_path + '/' + categorie + '/' + id_image
    print('Image :', lb_image)

    # Lecture et normalisation de l'image
    img = load_img(lb_image, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = img_to_array(img, dtype=np.uint8)
    img_array = np.array(img_array)/255.0
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    # Affichage
    img = cv2.imread(lb_image)
    plt.figure(figsize=(4,4))
    print(plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)))
    plt.show()

    # Prédiction et évaluation
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    # Affichage des résultats
    print("Cette image appartient probablement à la classe {} avec un niveau de confiance à {:.2f}."
        .format(lst_labels[np.argmax(score)], 100 * np.max(score)))

    print(tf.nn.softmax(predictions).numpy())
    return predictions, score

In [ ]:
def print_layer_trainable(model):
    ''' Statut et nom des couches d'un modèle
    '''
    for layer in model.layers:
        print("{0}:\t{1}".format(layer.trainable, layer.name))

# Construction du modèle et apprentissage des données